# Finetune ruGPT3Small on your data


## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## Add data to colab


In [ ]:
!wget https://raw.githubusercontent.com/FurkanArslan/twitter-text-classification/master/twitter_sentiment_corpus.csv

In [ ]:
data_path = "drive/MyDrive/LD final/vacancy_all_it.csv"

## Prepare data

In [ ]:
import pandas as pd
data = pd.read_csv(data_path, index_col=0)
data.head()

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
data.shape

In [ ]:
val_ind = random.sample(range(data.shape[0]), 500)

In [ ]:
train = [data.iloc[i]['description'] for i in range(len(data)) if i not in val_ind][:10000]
valid = [data.iloc[i]['description'] for i in range(len(data)) if i in val_ind]

In [ ]:
len(train), len(valid)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain_transformers.py \
    --output_dir=comment_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

## Check our model

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --k=5 \
    --p=0.95 \
    --length=75